<a href="https://colab.research.google.com/github/gagan3012/keytotext/blob/master/modeltexttokey.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone https://gitlab.com/shimorina/webnlg-dataset.git
!pip install transformers
!pip install sentencepiece


Cloning into 'webnlg-dataset'...
remote: Enumerating objects: 5106, done.
remote: Total 5106 (delta 0), reused 0 (delta 0), pack-reused 5106
Receiving objects: 100% (5106/5106), 26.08 MiB | 21.08 MiB/s, done.
Resolving deltas: 100% (4008/4008), done.
Checking out files: 100% (1425/1425), done.
     |████████████████████████████████| 1.9MB 8.0MB/s 
     |████████████████████████████████| 890kB 38.9MB/s 
     |████████████████████████████████| 3.2MB 48.8MB/s 
  Created wheel for sacremoses: filename=sacremoses-0.0.43-cp37-none-any.whl size=893262 sha256=88375fd6ec9a0ca9fd70f825a4dfeba4ccbdbfc08a2dedd7d928e2b39d9f77ec
  Stored in directory: /root/.cache/pip/wheels/29/3c/fd/7ce5c3f0666dab31a50123635e6fb5e19ceb42ce38d4e58f45
Successfully built sacremoses
     |████████████████████████████████| 1.2MB 8.6MB/s 


In [2]:
import pandas as pd
import torch
import os
from transformers import T5Tokenizer, T5ForConditionalGeneration,Adafactor

In [3]:
import glob
import os
import re
import xml.etree.ElementTree as ET
import pandas as pd
files = glob.glob("/content/webnlg-dataset/release_v3.0/en/train/**/*.xml", recursive=True)
triple_re=re.compile('(\d)triples')
data_dct={}
for file in files:
    tree = ET.parse(file)
    root = tree.getroot()
    triples_num=int(triple_re.findall(file)[0])
    for sub_root in root:
        for ss_root in sub_root:
            strutured_master=[]
            unstructured=[]
            for entry in ss_root:
                unstructured.append(entry.text)
                strutured=[triple.text for triple in entry]
                strutured_master.extend(strutured)
            unstructured=[i for i in unstructured if i.replace('\n','').strip()!='' ]
            strutured_master=strutured_master[-triples_num:]
            strutured_master_str=(' && ').join(strutured_master)
            data_dct[strutured_master_str]=unstructured
mdata_dct={"prefix":[], "input_text":[], "target_text":[]}
for st,unst in data_dct.items():
    for i in unst:
        mdata_dct['prefix'].append('webNLG')
        mdata_dct['input_text'].append(st)
        mdata_dct['target_text'].append(i)


df=pd.DataFrame(mdata_dct)
df.to_csv('webNLG2020.csv')
df

,prefix,input_text,target_text
0,webNLG,"1_Decembrie_1918_University | latinName | ""Uni...",1 Decembrie 1918 University is located in Alba...
1,webNLG,"1_Decembrie_1918_University | latinName | ""Uni...",The 1 Decembrie 1918 University is located in ...
2,webNLG,"1_Decembrie_1918_University | latinName | ""Uni...",The Latin name of 1 Decembrie 1918 University ...
3,webNLG,1_Decembrie_1918_University | nickname | Uab &...,The 1 Decembrie 1918 University is located in ...
4,webNLG,1_Decembrie_1918_University | nickname | Uab &...,The nickname of the 1 Decembrie 1918 Universit...
...,...,...,...
35197,webNLG,Trane | product | HVAC && Trane | industry | B...,"The Trane industry is building materials, and ..."
35198,webNLG,Trane | product | HVAC && Trane | industry | B...,Trane is in the building materials industry an...
35199,webNLG,Trane | revenue | 10264000000 && Trane | locat...,"With a revenue of $10,264,000,000, Trane is lo..."
35200,webNLG,Trane | revenue | 10264000000 && Trane | locat...,"The location of Trane is Swords, Dublin and ha..."


In [4]:
train_df=pd.read_csv('webNLG2020.csv', index_col=[0])
train_df=train_df.iloc[  :35202,:]
train_df=train_df.sample(frac = 1)
batch_size=8
num_of_batches=int(len(train_df)/batch_size)

In [5]:
num_of_batches

4400

In [6]:
train_df.head()

,prefix,input_text,target_text
17242,webNLG,Juan_Perón | party | Labour_Party_(Argentina) ...,"Alberto Teisaire, born in Mendoza, Argentina, ..."
11743,webNLG,Italy | capital | Rome,The capital of Italy is Rome.
7831,webNLG,"Anderson_Township,_Madison_County,_Indiana | c...","Anderson, Township of Madison County, Indiana ..."
7548,webNLG,Hypermarcas | type | S.A._(corporation) && Hyp...,"Hypermarcas, a type of S.A. corporation, has a..."
34469,webNLG,Agremiação_Sportiva_Arapiraquense | ground | E...,Agremiacao Sportiva Arapiraquense's ground is ...


In [7]:
if torch.cuda.is_available():
    dev = torch.device("cuda:0")
    print("Running on the GPU")
else:
    dev = torch.device("cpu")
    print("Running on the CPU")

Running on the GPU


In [8]:
from IPython.display import HTML, display
def progress(loss,value, max=100):
    return HTML(""" Batch loss :{loss}
      <progress    
value='{value}'max='{max}',style='width: 100%'>{value}
      </progress>
             """.format(loss=loss,value=value, max=max))

In [9]:
tokenizer = T5Tokenizer.from_pretrained('t5-base')
model = T5ForConditionalGeneration.from_pretrained('t5-base',
                                             return_dict=True)
#moving the model to GPU
model.to(dev)

T5ForConditionalGeneration(
  (shared): Embedding(32128, 768)
  (encoder): T5Stack(
    (embed_tokens): Embedding(32128, 768)
    (block): ModuleList(
      (0): T5Block(
        (layer): ModuleList(
          (0): T5LayerSelfAttention(
            (SelfAttention): T5Attention(
              (q): Linear(in_features=768, out_features=768, bias=False)
              (k): Linear(in_features=768, out_features=768, bias=False)
              (v): Linear(in_features=768, out_features=768, bias=False)
              (o): Linear(in_features=768, out_features=768, bias=False)
              (relative_attention_bias): Embedding(32, 12)
            )
            (layer_norm): T5LayerNorm()
            (dropout): Dropout(p=0.1, inplace=False)
          )
          (1): T5LayerFF(
            (DenseReluDense): T5DenseReluDense(
              (wi): Linear(in_features=768, out_features=3072, bias=False)
              (wo): Linear(in_features=3072, out_features=768, bias=False)
              (dropout): Dr

In [11]:

optimizer = Adafactor(model.parameters(),lr=1e-3,
                      eps=(1e-30, 1e-3),
                      clip_threshold=1.0,
                      decay_rate=-0.8,
                      beta1=None,
                      weight_decay=0.0,
                      relative_step=False,
                      scale_parameter=False,
                      warmup_init=False)

In [ ]:
model.train()
num_of_epochs = 5

loss_per_10_steps=[]
for epoch in range(1,num_of_epochs+1):
    print('Running epoch: {}'.format(epoch))
  
    running_loss=0

    out = display(progress(1, num_of_batches+1), display_id=True)
    for i in range(num_of_batches):
        inputbatch=[]
        labelbatch=[]
        new_df=train_df[i*batch_size:i*batch_size+batch_size]
        for indx,row in new_df.iterrows():
            input = 'WebNLG: '+row['input_text']+'</s>' 
            labels = row['target_text']+'</s>'   
            inputbatch.append(input)
            labelbatch.append(labels)
        inputbatch=tokenizer.batch_encode_plus(inputbatch,padding=True,max_length=400,return_tensors='pt')["input_ids"]
        labelbatch=tokenizer.batch_encode_plus(labelbatch,padding=True,max_length=400,return_tensors="pt") ["input_ids"]
        inputbatch=inputbatch.to(dev)
        labelbatch=labelbatch.to(dev)

        # clear out the gradients of all Variables 
        optimizer.zero_grad()

        # Forward propogation
        outputs = model(input_ids=inputbatch, labels=labelbatch)
        loss = outputs.loss
        loss_num=loss.item()
        logits = outputs.logits
        running_loss+=loss_num
        if i%10 ==0:      
            loss_per_10_steps.append(loss_num)
        out.update(progress(loss_num,i, num_of_batches+1))

        # calculating the gradients
        loss.backward()

        #updating the params
        optimizer.step()
    
    running_loss=running_loss/int(num_of_batches)
    print('Epoch: {} , Running loss: {}'.format(epoch,running_loss))


Running epoch: 1


/usr/local/lib/python3.7/dist-packages/transformers/models/t5/tokenization_t5.py:186: UserWarning: This sequence already has </s>. In future versions this behavior may lead to duplicated eos tokens being added.
  f"This sequence already has {self.eos_token}. In future versions this behavior may lead to duplicated eos tokens being added."
/usr/local/lib/python3.7/dist-packages/transformers/optimization.py:557: UserWarning: This overload of add_ is deprecated:
	add_(Number alpha, Tensor other)
Consider using one of the following signatures instead:
	add_(Tensor other, *, Number alpha) (Triggered internally at  /pytorch/torch/csrc/utils/python_arg_parser.cpp:1005.)
  exp_avg_sq_row.mul_(beta2t).add_(1.0 - beta2t, update.mean(dim=-1))
